In [1]:
%autoreload 2


In [2]:
from argparse import Namespace
from collections import defaultdict
import copy
import difflib
import gzip
import itertools
import os
import pickle
import sys
import typing

import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

from IPython.display import display, Markdown, HTML
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline
import tatsu
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm.notebook as tqdm


sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.ast_counter_sampler import *
from src.ast_utils import cached_load_and_parse_games_from_file, load_games_from_file, _extract_game_id
from src import ast_printer

2023-09-22 11:53:57 - ast_utils - DEBUG    - Using cache folder: /misc/vlgscratch4/LakeGroup/guy/game_generation_cache
2023-09-22 11:53:57 - src.ast_utils - DEBUG    - Using cache folder: /misc/vlgscratch4/LakeGroup/guy/game_generation_cache


In [3]:
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='..'))
real_game_texts = [ast_printer.ast_to_string(ast, '\n') for ast in game_asts]
regrown_game_1024_texts = list(load_games_from_file('../dsl/ast-real-regrowth-samples-1024.pddl.gz'))
print(len(real_game_texts), len(regrown_game_1024_texts), len(regrown_game_1024_texts) / 98)


2023-09-22 11:53:59 - src.ast_utils - INFO     - Loading from cache file: /misc/vlgscratch4/LakeGroup/guy/game_generation_cache/interactive-beta-cache.pkl.gz
2023-09-22 11:53:59 - src.ast_utils - INFO     - Finished loading cache file: /misc/vlgscratch4/LakeGroup/guy/game_generation_cache/interactive-beta-cache.pkl.gz


98 100352 1024.0


In [4]:
def extract_game_index(game_name: str):
    first_dash = game_name.find('-')
    second_dash = game_name.find('-', first_dash + 1)
    index = game_name[first_dash + 1:second_dash] if second_dash != -1 else game_name[first_dash + 1:]
    return int(index)


def extract_negative_index(game_name: str):
    first_dash = game_name.find('-')
    second_dash = game_name.find('-', first_dash + 1)
    if second_dash == -1:
        return -1
    
    third_dash = game_name.find('-', second_dash + 1)
    index = game_name[second_dash + 1:third_dash]
    return int(index)


fitness_df = utils.load_fitness_data('../data/fitness_features_1024_regrowths.csv.gz')

# fitness_df = fitness_df.assign(real=fitness_df.real.astype('int'), game_index=fitness_df.game_name.apply(extract_game_index), 
#                                negative_index= fitness_df.game_name.apply(extract_negative_index), fake=~fitness_df.real.astype('int'))
# fitness_df = fitness_df.sort_values(by=['fake', 'game_index', 'negative_index'], ignore_index=True).reset_index(drop=True)
# fitness_df.drop(columns=['Index', 'fake', 'game_index', 'negative_index'], inplace=True)
print(fitness_df.src_file.unique())
fitness_df.head()

['interactive-beta.pddl' 'ast-real-regrowth-samples-1024.pddl.gz']


,Index,src_file,game_name,domain_name,original_game_name,real,variables_defined_all,variables_defined_prop,variables_used_all,variables_used_prop,...,ast_ngram_constraints_n_4_score,ast_ngram_constraints_n_5_score,ast_ngram_terminal_n_2_score,ast_ngram_terminal_n_3_score,ast_ngram_terminal_n_4_score,ast_ngram_terminal_n_5_score,ast_ngram_scoring_n_2_score,ast_ngram_scoring_n_3_score,ast_ngram_scoring_n_4_score,ast_ngram_scoring_n_5_score
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,6172feb1665491d1efbce164-0,1,1,1.0,1,1.0,...,0.968048,0.979128,0.968876,0.979584,0.980420,0.982740,0.919068,0.960887,0.998712,1.000000
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,5f77754ba932fb2c4ba181d8-2,1,1,1.0,1,1.0,...,0.973823,0.978192,0.965107,0.976486,0.975603,0.983432,0.899002,0.914971,0.965387,0.974323
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,614b603d4da88384282967a7-3,1,1,1.0,1,1.0,...,0.945618,0.972216,0.000000,0.000000,0.000000,0.000000,0.850539,0.904462,0.948491,0.957243
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,5bc79f652885710001a0e82a-5,1,1,1.0,1,1.0,...,0.986638,0.980074,0.000000,0.000000,0.000000,0.000000,0.919068,0.960887,0.998712,1.000000
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,614dec67f6eb129c3a77defd-6,1,1,1.0,1,1.0,...,0.982992,0.979321,0.000000,0.000000,0.000000,0.000000,0.964519,0.971914,0.973287,0.964638


In [5]:
from latest_model_paths import LATEST_FITNESS_FUNCTION_DATE_ID
# Change here if you want to use a different model
model_date_id = LATEST_FITNESS_FUNCTION_DATE_ID
data_df = fitness_df
cv_energy_model, feature_columns = utils.load_model_and_feature_columns(model_date_id)
print(len(feature_columns))


87


In [6]:
full_tensor = utils.df_to_tensor(data_df, feature_columns)
if 'wrapper' in cv_energy_model.named_steps: cv_energy_model.named_steps['wrapper'].eval()
full_tensor_scores = cv_energy_model.transform(full_tensor).detach()

real_game_scores = full_tensor_scores[:, 0]

print(f'Real game scores: {real_game_scores.mean():.4f} ± {real_game_scores.std():.4f}, min = {real_game_scores.min():.4f}, max = {real_game_scores.max():.4f}')

negatives_scores = full_tensor_scores[:, 1:]
torch.quantile(negatives_scores.ravel(), torch.linspace(0, 1, 11))
print(f'20th percentile negative energy: {torch.quantile(negatives_scores.ravel(), 0.2)}')

Real game scores: -89.7050 ± 0.8883, min = -91.7563, max = -87.9750
20th percentile negative energy: -86.12468719482422


In [7]:
steps = torch.linspace(0, 1, 11)
deciles = torch.quantile(negatives_scores.ravel(), steps)
print(steps)
print(f'Energy deciles: {deciles}')

tensor([0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
        0.9000, 1.0000])
Energy deciles: tensor([-91.7563, -87.5674, -86.1247, -84.6908, -83.3112, -82.1262, -80.6836,
        -78.7567, -76.0111, -70.2416, -25.4719])


In [8]:
def print_negatives_from_quantile(quantile: int, n_games: int, output_dir: str = './temp_outputs/'):
    min_energy = deciles[2 * quantile]
    max_energy = deciles[2 * quantile + 1]
    quantile_indices = ((negatives_scores >= min_energy) & (negatives_scores <= max_energy)).nonzero()
    game_indices = torch.randperm(quantile_indices.shape[0])[:n_games]
    lines = [f'; Games from quantile #{quantile} with energies in the range [{min_energy:.4f} ({int(steps[2 * quantile] * 100)}%), {max_energy:.4f} ({int(steps[2 * (quantile + 1)] * 100)}%) ]']
    for idx in game_indices:
        overall_index = 98 * quantile_indices[idx, 0] + quantile_indices[idx, 1]
        lines.append(f'; Game with index {overall_index} and energy {negatives_scores[quantile_indices[idx, 0], quantile_indices[idx, 1], quantile_indices[idx, 2]]:.4f}')
        lines.append(regrown_game_1024_texts[overall_index])
        lines.append('')

    with open(os.path.join(output_dir, f'quantile_{quantile}.txt'), 'w') as f:
        f.write('\n'.join(lines))


# n_games_to_print = 25
# for q in range(5):
#     print_negatives_from_quantile(q, n_games_to_print)
    


In [9]:
RANDOM_SEED = 33
torch.manual_seed(RANDOM_SEED)

N_GAMES = 20
N_GAMES_PER_QUANTILE = 10

original_game_indices = list(torch.randperm(full_tensor.shape[0])[:N_GAMES].numpy())
regrowths_by_original_game_index_and_quantile = {idx: {} for idx in original_game_indices}

for game_idx in original_game_indices:
    for q in range(5):
        min_energy = deciles[2 * q]
        max_energy = deciles[2 * q + 1]
        game_negatives = negatives_scores[game_idx].squeeze()
        quantile_indices = ((game_negatives >= min_energy) & (game_negatives <= max_energy)).nonzero().squeeze()
        negatives_in_quantile_indices = torch.randperm(quantile_indices.shape[0])[:N_GAMES_PER_QUANTILE]
        regrowths_by_original_game_index_and_quantile[game_idx][q] = list(quantile_indices[negatives_in_quantile_indices].numpy())

        

In [10]:
def visualize_game_diff(game_idx: int, quantile: int, negative_idx: int, output_dir: str = './temp_outputs/'):
    original_game_index = original_game_indices[game_idx]
    original_game = real_game_texts[original_game_index]
    regrowth_game = regrown_game_1024_texts[original_game_index * 1024 + negative_idx]

    utils.display_game_diff_html(original_game, regrowth_game)


game_index = 0  # 0 to N_GAMES - 1
quantile = 0  # 0 to 4
negative_index = 0  # 0 to N_GAMES_PER_QUANTILE - 1

visualize_game_diff(game_index, quantile, negative_index)
    

n,1,(define (game 6158d01f302cf46b673dd597-87) (:domain few-objects-room-v1),n,1,(define (game 6158d01f302cf46b673dd597-87-0-nd-17-rd-6-rs-constraints-sk-prior22
,,,,>,-nr-1) (:domain few-objects-room-v1)
,2,(:setup,,2,(:setup
,3,(and,,3,(and
,4,(exists (?h - hexagonal_bin ?w - wall),,4,(exists (?h - hexagonal_bin ?w - wall)
,5,(game-conserved,,5,(game-conserved
,6,(and,,6,(and
,7,(on desk ?h),,7,(on desk ?h)
,8,(adjacent ?h ?w),,8,(adjacent ?h ?w)
,9,),,9,)
,10,),,10,)
